In [1]:
%%capture
%cd ../../
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import logging

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
from tqdm import tqdm
from xgboost import XGBClassifier
import pandas as pd

from ml_common.constants import SYMP_COLS, SYMP_CHANGE_COLS
from ml_common.filter import (
    drop_highly_missing_features, 
    drop_samples_with_no_targets, 
    drop_unused_drug_features,
    drop_samples_outside_study_date, 
    keep_only_one_per_week
)
from ml_common.engineer import collapse_rare_categories, get_change_since_prev_session, get_missingness_features
from ml_common.summary import get_label_distribution
from ml_common.util import get_nunique_categories, get_nmissing

from preduce.symp.label import convert_to_binary_symptom_labels, get_symptom_labels
from preduce.symp.pipeline import PrepSympData
from preduce.prepare.prep import fill_missing_data
from preduce.summarize import feature_summary, get_patient_characteristics
from preduce.util import initialize_folders

from sklearn.exceptions import ConvergenceWarning
import warnings

pd.set_option('display.max_rows', 150)

warnings.filterwarnings("ignore", category=ConvergenceWarning)

initialize_folders()

logging.basicConfig(
    filename=f"./logs/{datetime.now().strftime('%Y-%m-%d %H.%M.%S')}_symptom_target.log",
    level=logging.INFO, 
    format='%(asctime)s %(levelname)s:%(message)s', 
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [3]:
# Load data
df = pd.read_parquet('data/processed/treatment_centered_clinical_dataset.parquet.gzip')
df['assessment_date'] = df['treatment_date']

In [4]:
# scoring increase thresholds for determining symptom deterioration
target_pt_increases = [1, 3]

# Prep Data

In [5]:
# filter out dates before 2014 and after 2020
df = drop_samples_outside_study_date(df)

# keep only the first treatment session of a given week
df = keep_only_one_per_week(df)

# get the change in measurement since previous assessment
df = get_change_since_prev_session(df)

# extract labels
symp = pd.read_parquet('./data/interim/symptom.parquet.gzip')
df = get_symptom_labels(df, symp)
for pt_increase in target_pt_increases:
    scoring_map = {symp: pt_increase for symp in SYMP_COLS}
    df = convert_to_binary_symptom_labels(df, scoring_map=scoring_map)

# filter out sessions without any labels
target_cols = 'target_' + pd.Index(SYMP_CHANGE_COLS)
df = drop_samples_with_no_targets(df, target_cols)

# drop drug features that were never used
df = drop_unused_drug_features(df)

Getting symptom labels...: 100%|██████████| 1097/1097 [00:06<00:00, 163.31it/s]


In [6]:
get_nunique_categories(df)

,regimen,intent
Number of Unique Categories,107,4


In [7]:
nmissing = get_nmissing(df)
nmissing[~nmissing.index.str.endswith('_date')].tail(20)

,Missing (N),Missing (%)
monocyte_change,14311,62.737
albumin_change,14450,63.347
basophil_change,14752,64.671
lactate_dehydrogenase_change,14927,65.438
esas_nausea_change,15220,66.722
alanine_aminotransferase_change,16140,70.755
bicarbonate,17600,77.156
basophil,18230,79.918
esas_appetite_change,19729,86.489
esas_drowsiness_change,20886,91.561


In [6]:
# fill missing data that can be filled heuristically
df = fill_missing_data(df)

# drop features with high missingness
keep_cols = df.columns[df.columns.str.contains('target_')]
df = drop_highly_missing_features(df, missing_thresh=75, keep_cols=keep_cols)

# create missingness features
df = get_missingness_features(df)

# collapse rare morphology and cancer sites into 'Other' category
df = collapse_rare_categories(df, catcols=['cancer_site', 'morphology'])

In [7]:
prep = PrepSympData()
X, Y, metainfo = prep.run_pipeline(df, split_date='2017-09-30', target_pt_increases=target_pt_increases)
df = df.loc[X.index]
# clean up Y
Y = Y[[col for col in Y.columns if col.endswith('pt_change')]]
for substr in ['target_', 'esas_']: Y.columns = Y.columns.str.replace(substr, '')

In [8]:
train_mask, valid_mask, test_mask = metainfo['split'] == 'Train', metainfo['split'] == 'Valid', metainfo['split'] == 'Test'
X_train, X_valid, X_test = X[train_mask], X[valid_mask], X[test_mask]
Y_train, Y_valid, Y_test = Y[train_mask], Y[valid_mask], Y[test_mask]

# Describe Data

In [16]:
count = pd.DataFrame({
    'Number of sessions': metainfo.groupby('split').apply(len, include_groups=False), 
    'Number of patients': metainfo.groupby('split')['mrn'].nunique()}
).T
count['Total'] = count.sum(axis=1)
print(f'\n{count.to_string()}')


split               Test  Train  Valid  Total
Number of sessions  5570  11723   2974  20267
Number of patients   960   1815    454   3229


In [17]:
get_label_distribution(Y, metainfo, with_respect_to='sessions').sort_index()

Test             Train            Valid        \
                                   0     1   -1      0     1   -1     0     1   
anxiety_1pt_change              3854  1662   54   8137  3392  194  2089   832   
anxiety_3pt_change              4947   417  206  10281   919  523  2628   221   
appetite_1pt_change             3663  1817   90   7464  4066  193  1938   988   
appetite_3pt_change             4525   752  293   9531  1580  612  2431   389   
depression_1pt_change           4009  1519   42   8283  3270  170  2100   839   
depression_3pt_change           5032   391  147  10438   889  396  2673   224   
drowsiness_1pt_change           3438  2099   33   7198  4374  151  1855  1068   
drowsiness_3pt_change           4599   756  215   9714  1559  450  2470   389   
nausea_1pt_change               4031  1496   43   8202  3367  154  2113   839   
nausea_3pt_change               4974   502   94  10305  1161  257  2654   276   
pain_1pt_change                 3861  1686   23   7897  3694  132  2058   883   
pain_3pt_change                 4834   570  166  10026  1296  401  2597   295   
shortness_of_breath_1pt_change  3976  1548   46   8148  3435  140  2106   845   
shortness_of_breath_3pt_change  4918   419  233  10418   938  367  2684   220   
tiredness_1pt_change            3155  2376   39   6614  4927  182  1708  1228   
tiredness_3pt_change            4417   746  407   9268  1617  838  2407   376   
well_being_1pt_change           3337  2190   43   7021  4397  305  1868  1032   
well_being_3pt_change           4668   644  258   9695  1355  673  2512   309   

                                     Total              
                                 -1      0     1    -1  
anxiety_1pt_change               53  14080  5886   301  
anxiety_3pt_change              125  17856  1557   854  
appetite_1pt_change              48  13065  6871   331  
appetite_3pt_change             154  16487  2721  1059  
depression_1pt_change            35  14392  5628   247  
depression_3pt_change            77  18143  1504   620  
drowsiness_1pt_change            51  12491  7541   235  
drowsiness_3pt_change           115  16783  2704   780  
nausea_1pt_change                22  14346  5702   219  
nausea_3pt_change                44  17933  1939   395  
pain_1pt_change                  33  13816  6263   188  
pain_3pt_change                  82  17457  2161   649  
shortness_of_breath_1pt_change   23  14230  5828   209  
shortness_of_breath_3pt_change   70  18020  1577   670  
tiredness_1pt_change             38  11477  8531   259  
tiredness_3pt_change            191  16092  2739  1436  
well_being_1pt_change            74  12226  7619   422  
well_being_3pt_change           153  16875  2308  1084

In [18]:
get_label_distribution(Y, metainfo, with_respect_to='patients').sort_index()

Test      Train       Valid      Total      
                                  1    0     1     0     1    0     1     0
anxiety_1pt_change              611  349  1187   628   300  154  2098  1131
anxiety_3pt_change              234  726   510  1305   125  329   869  2360
appetite_1pt_change             636  324  1333   482   328  126  2297   932
appetite_3pt_change             386  574   777  1038   187  267  1350  1879
depression_1pt_change           569  391  1160   655   291  163  2020  1209
depression_3pt_change           226  734   497  1318   126  328   849  2380
drowsiness_1pt_change           723  237  1408   407   349  105  2480   749
drowsiness_3pt_change           403  557   785  1030   200  254  1388  1841
nausea_1pt_change               554  406  1173   642   283  171  2010  1219
nausea_3pt_change               276  684   609  1206   141  313  1026  2203
pain_1pt_change                 643  317  1290   525   301  153  2234   995
pain_3pt_change                 317  643   668  1147   167  287  1152  2077
shortness_of_breath_1pt_change  554  406  1165   650   283  171  2002  1227
shortness_of_breath_3pt_change  241  719   507  1308   119  335   867  2362
tiredness_1pt_change            777  183  1488   327   377   77  2642   587
tiredness_3pt_change            404  556   820   995   194  260  1418  1811
well_being_1pt_change           736  224  1420   395   354  100  2510   719
well_being_3pt_change           351  609   707  1108   171  283  1229  2000

In [19]:
# Feature Characteristics
x = prep.ohe.encode(df.loc[X_train.index].copy(), verbose=False) # get original (non-normalized, non-imputed) data one-hot encoded
x = x[[col for col in x.columns if not (col in metainfo.columns or col.startswith('target'))]]
feature_summary(x, save_path='result/tables/feature_summary.csv').sample(10, random_state=42)

,Features,Group,Mean (SD),Missingness (%)
141,Phosphate Change,Laboratory,0.059 (1.543),17.2
211,Regimen TRIAL,Treatment,0.084 (0.277),0.0
217,Intent of Systemic Treatment OTHER,Treatment,0.000 (0.000),0.0
175,Regimen GI-PANITUMUMAB,Treatment,0.011 (0.104),0.0
48,"Morphology ICD-0-3 848, Cystic, mucinous, and ...",Cancer,0.041 (0.198),0.0
105,Percentage of Ideal Dose Given RAMUCIRUMAB,Treatment,1.022 (0.038),0.0
167,Regimen GI-GEM 7-WEEKLY,Treatment,0.004 (0.066),0.0
72,Eosinophil (x10e9/L),Laboratory,0.175 (0.215),55.7
204,Regimen LU-PACLICARBO,Treatment,0.004 (0.060),0.0
215,Intent of Systemic Treatment Palliative,Treatment,0.732 (0.443),0.0


In [14]:
# Patient Characteristics
cancer_cols = [col for col in df.columns if col.startswith('cancer_site')]
top_cancers = df.loc[train_mask, cancer_cols].sum().sort_values(ascending=False)[:5].index
top_regimens = df.loc[train_mask, 'regimen'].value_counts()[:5].index
result = dict()
result[('Dev', 'Treatments')] = get_patient_characteristics(df[train_mask | valid_mask], top_regimens, top_cancers)
result[('Dev', 'Patients')] = get_patient_characteristics(df[train_mask | valid_mask].groupby('mrn').last(), top_regimens, top_cancers)
result[('Test', 'Treatments')] = get_patient_characteristics(df[test_mask], top_regimens, top_cancers)
result[('Test', 'Patients')] = get_patient_characteristics(df[test_mask].groupby('mrn').last(), top_regimens, top_cancers)
pd.DataFrame(result)

Dev  \
                                                  Treatments   
Number of Treatments, Median (IQR)                   4 (2-8)   
Age (years), Median (IQR)                         64 (56-70)   
Height (cm), Median (IQR)                168.0 (161.0-175.0)   
Weight (kg), Median (IQR)                   69.3 (59.1-81.6)   
Female, No. (%)                                  6304 (42.9)   
Regimen GI-FOLFIRI+BEVACIZUMAB, No. (%)          1607 (10.9)   
Regimen GI-FOLFIRINOX, No. (%)                    1239 (8.4)   
Regimen TRIAL, No. (%)                            1306 (8.9)   
Regimen GI-GEM D1,8,15, No. (%)                    982 (6.7)   
Regimen GI-FOLFOX-6 MOD, No. (%)                   954 (6.5)   
Cancer Site Bronchus and lung, No. (%)           3899 (26.5)   
Cancer Site Pancreas, No. (%)                    3391 (23.1)   
Cancer Site Colon, No. (%)                       1685 (11.5)   
Cancer Site Rectum, No. (%)                       1014 (6.9)   
Cancer Site Stomach, No. (%)                       923 (6.3)   

                                                              \
                                                    Patients   
Number of Treatments, Median (IQR)                   1 (1-1)   
Age (years), Median (IQR)                         64 (56-70)   
Height (cm), Median (IQR)                168.0 (161.0-175.0)   
Weight (kg), Median (IQR)                   69.2 (58.8-81.5)   
Female, No. (%)                                   948 (41.8)   
Regimen GI-FOLFIRI+BEVACIZUMAB, No. (%)            124 (5.5)   
Regimen GI-FOLFIRINOX, No. (%)                     138 (6.1)   
Regimen TRIAL, No. (%)                             167 (7.4)   
Regimen GI-GEM D1,8,15, No. (%)                    124 (5.5)   
Regimen GI-FOLFOX-6 MOD, No. (%)                   152 (6.7)   
Cancer Site Bronchus and lung, No. (%)            731 (32.2)   
Cancer Site Pancreas, No. (%)                     382 (16.8)   
Cancer Site Colon, No. (%)                         189 (8.3)   
Cancer Site Rectum, No. (%)                        113 (5.0)   
Cancer Site Stomach, No. (%)                       146 (6.4)   

                                                        Test  \
                                                  Treatments   
Number of Treatments, Median (IQR)                   4 (2-7)   
Age (years), Median (IQR)                         64 (57-71)   
Height (cm), Median (IQR)                169.0 (162.0-177.0)   
Weight (kg), Median (IQR)                   71.6 (60.7-83.4)   
Female, No. (%)                                  2183 (39.2)   
Regimen GI-FOLFIRI+BEVACIZUMAB, No. (%)            198 (3.6)   
Regimen GI-FOLFIRINOX, No. (%)                      41 (0.7)   
Regimen TRIAL, No. (%)                            865 (15.5)   
Regimen GI-GEM D1,8,15, No. (%)                    163 (2.9)   
Regimen GI-FOLFOX-6 MOD, No. (%)                   256 (4.6)   
Cancer Site Bronchus and lung, No. (%)           1936 (34.8)   
Cancer Site Pancreas, No. (%)                    1193 (21.4)   
Cancer Site Colon, No. (%)                         443 (8.0)   
Cancer Site Rectum, No. (%)                        214 (3.8)   
Cancer Site Stomach, No. (%)                       512 (9.2)   

                                                              
                                                    Patients  
Number of Treatments, Median (IQR)                   1 (1-1)  
Age (years), Median (IQR)                         65 (57-71)  
Height (cm), Median (IQR)                169.0 (162.0-176.0)  
Weight (kg), Median (IQR)                   71.0 (60.0-82.9)  
Female, No. (%)                                   372 (38.8)  
Regimen GI-FOLFIRI+BEVACIZUMAB, No. (%)             11 (1.1)  
Regimen GI-FOLFIRINOX, No. (%)                       5 (0.5)  
Regimen TRIAL, No. (%)                            102 (10.6)  
Regimen GI-GEM D1,8,15, No. (%)                     18 (1.9)  
Regimen GI-FOLFOX-6 MOD, No. (%)                    49 (5.1)  
Cancer Site Bronchus and lung, No.

# Train Model - Quick and Dirty

In [22]:
targets = Y.columns
LR_params = {'C': 0.3, 'penalty': 'l2', 'class_weight': 'balanced', 'max_iter': 2000, 'random_state': 42} # 'solver': 'saga', 
LR_model = {target: LogisticRegression(**LR_params) for target in targets}
XGB_params = dict(n_estimators=100, max_depth=6, learning_rate=0.01, min_child_weight=6, random_state=42)
XGB_model = {target: XGBClassifier(**XGB_params) for target in targets}
for target in tqdm(targets):
    mask = Y_train[target] != -1
    LR_model[target].fit(X_train[mask], Y_train.loc[mask, target])
    XGB_model[target].fit(X_train[mask], Y_train.loc[mask, target])

100%|██████████| 18/18 [02:05<00:00,  6.98s/it]


In [23]:
def evaluate(model, X, Y):
    result = {}
    for target, label in Y.items():
        mask = label != -1
        # check model.classes_ to confirm prediction of positive label is at index 1
        pred = model[target].predict_proba(X[mask])[: ,1]
        auprc = average_precision_score(label[mask], pred)
        auroc = roc_auc_score(label[mask], pred)
        result[target] = {'AUPRC': auprc, 'AUROC': auroc}
    return pd.DataFrame(result).T

In [20]:
evaluate(LR_model, X_valid, Y_valid)

,AUPRC,AUROC
pain_1pt_change,0.475862,0.655851
tiredness_1pt_change,0.627286,0.678208
nausea_1pt_change,0.453932,0.671093
depression_1pt_change,0.401786,0.651748
anxiety_1pt_change,0.388799,0.617675
drowsiness_1pt_change,0.561692,0.685365
appetite_1pt_change,0.563517,0.700907
well_being_1pt_change,0.563339,0.680049
shortness_of_breath_1pt_change,0.408388,0.653472
pain_3pt_change,0.292664,0.730638


In [21]:
evaluate(LR_model, X_test, Y_test)

,AUPRC,AUROC
pain_1pt_change,0.464188,0.637326
tiredness_1pt_change,0.611927,0.673078
nausea_1pt_change,0.443644,0.671883
depression_1pt_change,0.384616,0.626949
anxiety_1pt_change,0.401322,0.610335
drowsiness_1pt_change,0.564664,0.675702
appetite_1pt_change,0.517947,0.663640
well_being_1pt_change,0.570521,0.665429
shortness_of_breath_1pt_change,0.368694,0.600363
pain_3pt_change,0.226392,0.675176


In [22]:
evaluate(XGB_model, X_valid, Y_valid)

,AUPRC,AUROC
pain_1pt_change,0.497675,0.683565
tiredness_1pt_change,0.620820,0.689137
nausea_1pt_change,0.498506,0.706147
depression_1pt_change,0.475036,0.707769
anxiety_1pt_change,0.434293,0.676690
drowsiness_1pt_change,0.558526,0.703425
appetite_1pt_change,0.584689,0.728845
well_being_1pt_change,0.550627,0.692945
shortness_of_breath_1pt_change,0.410443,0.650389
pain_3pt_change,0.299741,0.751617


In [19]:
evaluate(XGB_model, X_test, Y_test)

,AUPRC,AUROC
pain_1pt_change,0.479161,0.662992
tiredness_1pt_change,0.604046,0.666222
nausea_1pt_change,0.445009,0.675159
depression_1pt_change,0.399748,0.652901
anxiety_1pt_change,0.430365,0.644815
drowsiness_1pt_change,0.569400,0.683269
appetite_1pt_change,0.516838,0.676955
well_being_1pt_change,0.554816,0.666361
shortness_of_breath_1pt_change,0.405333,0.637055
pain_3pt_change,0.244697,0.700301
